In [1]:
import inspect
from pyscf import gto,scf
import numpy as np 
from numpy.linalg import norm as norm
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
import basis_set_exchange as bse
from pyscf.geomopt.berny_solver import optimize
ang2bohr=1.8897261246
bohr2ang=.5291772109
np.set_printoptions(precision=4)

In [7]:
import sys
sys.path.append("/home/administrator/Documenti/APDFT/prototyping/hessian/AAFF/Libs/")
from alch_deriv_NEW import alch_deriv
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FDcoeffs import *
from FcMole import FcM,FcM_like
%load_ext autoreload
%autoreload 1
import itertools
%autoreload from AP_class import DeltaV,rotate_matrix,parse_charge,alchemy_cphf_deriv,make_dP,rotate_grad,rotate_matrix

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
benz_atom=""" 
C         6.95191249e-13  2.61173060e+00 -1.67580194e-08 ;
H         2.83052087e-13  4.63941350e+00 -1.15976219e-08 ;
C         2.26182505e+00  1.30586530e+00 -2.37508246e-08 ; 
H         4.01784995e+00  2.31970675e+00 -9.84687205e-09 ; 
C         2.26182505e+00 -1.30586530e+00  8.26065094e-09 ; 
H         4.01784995e+00 -2.31970675e+00  5.83890508e-08 ; 
C        -8.45954456e-13 -2.61173060e+00 -1.55922267e-08 ; 
H         1.49445304e-12 -4.63941350e+00 -3.26774154e-08 ; 
C        -2.26182505e+00 -1.30586530e+00 -8.28266973e-09 ; 
H        -4.01784995e+00 -2.31970675e+00  1.87236306e-09 ; 
C        -2.26182505e+00  1.30586530e+00  1.68546675e-08 ; 
H        -4.01784995e+00  2.31970675e+00  3.31289175e-08 ; 
"""
Benz = gto.M(atom=benz_atom,unit='bohr',basis='def2-TZVP',symmetry=True) #basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
ldm=np.load('./benz_dm.npy')

In [35]:
BenzHF=scf.RHF(Benz)
BenzHF.scf()

converged SCF energy = -230.785209492568


-230.7852094925678

In [36]:
BenzHF.energy_nuc()

205.40289969315694

In [37]:
a1=FcM_like(Benz,fcs=[[0,2,4,6],[1,-1,-1,1]])
ae1=scf.RHF(a1)
ae1.scf()

bubu
converged SCF energy = -237.391363973175


-237.39136397317478

In [38]:
a2=FcM_like(Benz,fcs=[[0,2,4,6],[-1,1,1,-1]])
ae2=scf.RHF(a2)
ae2.scf()

bubu
converged SCF energy = -237.402452886355


-237.402452886355

In [ ]:
ae1.energy_nuc(),ae2.energy_nuc()

In [8]:
%time d1,d2,d3= alch_deriv(BenzHF,[[0],[1]])
d1,d2,d3

CPU times: user 262 ms, sys: 32.5 ms, total: 294 ms
Wall time: 109 ms


(-24.392820318889658, -0.0029486757670566057, 0.26021645259666754)

In [9]:
Benz.symmetry

True

In [10]:
from scipy.spatial.transform import Rotation as R
rv=-np.asarray([0,0,1])*np.pi*2/3
r = R.from_rotvec(rv)

## Symmetry operations for Benzene $D_{6h}$ axis 
I need to <br>
1) dP (i) -> rotate dP (0/1) <br>
2) af(i) = rotate af (0/1)<br>
3) dV(i) =rotate dV(0/1)<br>

In [11]:
pt=Benz.aoslice_by_atom()[0,-2]
rpt=Benz.aoslice_by_atom()[2,-2]

In [12]:
P=BenzHF.make_rdm1()
P[pt:pt+5,pt:pt+5],P[rpt:rpt+5,rpt:rpt+5]

(array([[ 2.0703e+00, -2.0748e-01, -4.6885e-16, -1.8160e-03,  0.0000e+00],
        [-2.0748e-01,  7.8356e-01,  4.9199e-15,  1.9056e-02,  0.0000e+00],
        [-4.6885e-16,  4.9199e-15,  6.0881e-01,  3.7133e-15,  0.0000e+00],
        [-1.8160e-03,  1.9056e-02,  3.7133e-15,  6.2319e-01,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  7.8543e-01]]),
 array([[ 2.0703e+00, -2.0748e-01, -1.5727e-03, -9.0799e-04,  0.0000e+00],
        [-2.0748e-01,  7.8356e-01,  1.6503e-02,  9.5282e-03,  0.0000e+00],
        [-1.5727e-03,  1.6503e-02,  6.1959e-01,  6.2279e-03,  0.0000e+00],
        [-9.0799e-04,  9.5282e-03,  6.2279e-03,  6.1240e-01,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  7.8543e-01]]))

In [13]:
np.cos(np.pi/3)*0.0191,np.sin(np.pi/3)*0.0191

(0.009550000000000001, 0.016541085212282775)

In [14]:
from pyscf import symm

In [15]:
BenzHF.get_irrep_nelec()

{'Ag': 12,
 'B1g': 6,
 'B2g': 2,
 'B3g': 2,
 'Au': 0,
 'B1u': 2,
 'B2u': 8,
 'B3u': 10}

In [16]:
%time BenzHF.Gradients().grad()

--------------- SymAdaptedRHF gradients ---------------
         x                y                z
0 C     0.0000000000     0.0029582667     0.0000000000
1 H    -0.0000000000    -0.0090174306     0.0000000000
2 C     0.0025619354     0.0014791328     0.0000000000
3 H    -0.0078093248    -0.0045087158     0.0000000000
4 C     0.0025619354    -0.0014791328     0.0000000000
5 H    -0.0078093248     0.0045087158     0.0000000000
6 C    -0.0000000000    -0.0029582667     0.0000000000
7 H     0.0000000000     0.0090174306     0.0000000000
8 C    -0.0025619354    -0.0014791328     0.0000000000
9 H     0.0078093248     0.0045087158     0.0000000000
10 C    -0.0025619354     0.0014791328     0.0000000000
11 H     0.0078093248    -0.0045087158     0.0000000000
----------------------------------------------
CPU times: user 5.28 s, sys: 78.6 ms, total: 5.36 s
Wall time: 3.79 s


array([[ 7.6376e-16,  2.9583e-03,  0.0000e+00],
       [-2.3281e-15, -9.0174e-03,  0.0000e+00],
       [ 2.5619e-03,  1.4791e-03,  0.0000e+00],
       [-7.8093e-03, -4.5087e-03,  0.0000e+00],
       [ 2.5619e-03, -1.4791e-03,  0.0000e+00],
       [-7.8093e-03,  4.5087e-03,  0.0000e+00],
       [-7.6376e-16, -2.9583e-03,  0.0000e+00],
       [ 2.3281e-15,  9.0174e-03,  0.0000e+00],
       [-2.5619e-03, -1.4791e-03,  0.0000e+00],
       [ 7.8093e-03,  4.5087e-03,  0.0000e+00],
       [-2.5619e-03,  1.4791e-03,  0.0000e+00],
       [ 7.8093e-03, -4.5087e-03,  0.0000e+00]])

In [17]:
coords=Benz.atom_coords()

In [18]:
#symm.Dmatrix.get_euler_angles(coords[0],coords[2])

In [19]:
#symm.geom.detect_symm(Benz.atom_and_coords())

In [20]:
dir(Benz)

['__add__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_suffix',
 '_atm',
 '_atom',
 '_bas',
 '_basis',
 '_build_symmetry',
 '_built',
 '_ecp',
 '_ecpbas',
 '_enuc',
 '_env',
 '_keys',
 '_libcint_ctr_coeff',
 '_nao',
 '_nelectron',
 '_pseudo',
 '_symm_axes',
 '_symm_orig',
 'ao2mo',
 'ao_labels',
 'ao_loc',
 'ao_loc_2c',
 'ao_loc_nr',
 'aoslice_2c_by_atom',
 'aoslice_by_atom',
 'aoslice_nr_by_atom',
 'apply',
 'atom',
 'atom_charge',
 'atom_charges',
 'atom_coord',
 'atom_coords',
 'atom_mass_list',
 'atom_nelec_core',
 'atom_nshells',
 'atom_pure_symbol',
 'atom_shell_ids',
 'atom_symbol',
 'bas_angular',
 'bas_a